In [2]:
import gym
import numpy as np
import random

##### A visual look at Q values for a grid world
https://docs.google.com/spreadsheets/d/1mgSpySJsBGZ3jp0m3xRpGxsfzs2I7z3pLs2yWJZlyOU/edit?usp=sharing

## Tabular Q
The formula of Q is: 

### $ Q(S_{t}, A_{t}) \leftarrow Q(S_{t}, A_{t}) + \alpha [R_{t+1} + \gamma \ \underset{a}{\operatorname{max}} Q(S_{t+1}, a) - Q(S_{t}, A_{t})] $

### Symbols
$S \to$ state  
$A \to$ action  
$Q(S_{t}, A_{t}) \to$ a function that takes parameters (s, a) and returns a Q value.  
$\alpha \to$ the learning rate, i.e. it adjusts how much of the new experience we store into $Q(S_{t}, A_{t})$  
$\gamma \to$ the discount rate, how much we discount future reward per time step  
$\underset{a}{\operatorname{max}} Q(S_{t+1}, a) \to$ call Q function with the $a$ that gives the highest value, i.e. given $S_{t}$, return the highest value between left and right action

### Note if we simplify the above and set $\alpha$ == 1
### $ Q(S_{t}, A_{t}) \leftarrow R_{t+1} + \gamma \ \underset{a}{\operatorname{max}} Q(S_{t+1}, a)$
In this scenario we update our Q value with our reward and the highest Q value of the next state


# Linear Frozen Lake environment

In [3]:
"""NO NEED TO CHANGE THIS CELL"""
import sys
from six import StringIO, b

from gym import utils
from gym.envs.toy_text import discrete

LEFT = 0
DOWN = 1
RIGHT = 2
UP = 3

MAPS = {
    "4x4": [
        "SFFF",
        "FHFH",
        "FFFH",
        "HFFG"
    ],
    "1x8": [
        "FFFSFFFG"
    ],
    "8x8": [
        "SFFFFFFF",
        "FFFFFFFF",
        "FFFHFFFF",
        "FFFFFHFF",
        "FFFHFFFF",
        "FHHFFFHF",
        "FHFFHFHF",
        "FFFHFFFG"
    ],
}

class FrozenLakeEnv(discrete.DiscreteEnv):
    """
    Winter is here. You and your friends were tossing around a frisbee at the park
    when you made a wild throw that left the frisbee out in the middle of the lake.
    The water is mostly frozen, but there are a few holes where the ice has melted.
    If you step into one of those holes, you'll fall into the freezing water.
    At this time, there's an international frisbee shortage, so it's absolutely imperative that
    you navigate across the lake and retrieve the disc.
    However, the ice is slippery, so you won't always move in the direction you intend.
    The surface is described using a grid like the following
        SFFF
        FHFH
        FFFH
        HFFG
    S : starting point, safe
    F : frozen surface, safe
    H : hole, fall to your doom
    G : goal, where the frisbee is located
    The episode ends when you reach the goal or fall in a hole.
    You receive a reward of 1 if you reach the goal, and zero otherwise.
    """

    metadata = {'render.modes': ['human', 'ansi']}

    def __init__(self, desc=None, map_name="4x4",is_slippery=True):
        if desc is None and map_name is None:
            raise ValueError('Must provide either desc or map_name')
        elif desc is None:
            desc = MAPS[map_name]
        self.desc = desc = np.asarray(desc,dtype='c')
        self.nrow, self.ncol = nrow, ncol = desc.shape
        self.reward_range = (0, 1)

        nA = 4
        nS = nrow * ncol

        isd = np.array(desc == b'S').astype('float64').ravel()
        isd /= isd.sum()

        P = {s : {a : [] for a in range(nA)} for s in range(nS)}

        def to_s(row, col):
            return row*ncol + col
        
        def inc(row, col, a):
            if a==0: # left
                col = max(col-1,0)
            elif a==1: # down
                row = min(row+1,nrow-1)
            elif a==2: # right
                col = min(col+1,ncol-1)
            elif a==3: # up
                row = max(row-1,0)
            return (row, col)

        for row in range(nrow):
            for col in range(ncol):
                s = to_s(row, col)
                for a in range(4):
                    li = P[s][a]
                    letter = desc[row, col]
                    if letter in b'GH':
                        li.append((1.0, s, 0, True))
                    else:
                        if is_slippery:
                            for b in [(a-1)%4, a, (a+1)%4]:
                                newrow, newcol = inc(row, col, b)
                                newstate = to_s(newrow, newcol)
                                newletter = desc[newrow, newcol]
                                done = bytes(newletter) in b'GH'
                                rew = float(newletter == b'G')
                                li.append((1.0/3.0, newstate, rew, done))
                        else:
                            newrow, newcol = inc(row, col, a)
                            newstate = to_s(newrow, newcol)
                            newletter = desc[newrow, newcol]
                            done = bytes(newletter) in b'GH'
                            rew = float(newletter == b'G')
                            li.append((1.0, newstate, rew, done))

        super(FrozenLakeEnv, self).__init__(nS, nA, P, isd)

    def render(self, mode='human'):
        outfile = StringIO() if mode == 'ansi' else sys.stdout

        row, col = self.s // self.ncol, self.s % self.ncol
        desc = self.desc.tolist()
        desc = [[c.decode('utf-8') for c in line] for line in desc]
        desc[row][col] = utils.colorize(desc[row][col], "red", highlight=True)
        if self.lastaction is not None:
            outfile.write("  ({})\n".format(["Left","Down","Right","Up"][self.lastaction]))
        else:
            outfile.write("\n")
        outfile.write("\n".join(''.join(line) for line in desc)+"\n")

        if mode != 'human':
            return outfile

        


# Environment Description

In [ ]:
"""
Winter is here. You and your friends were tossing around a frisbee at the park
when you made a wild throw that left the frisbee out in the middle of the lake.
The water is mostly frozen, but there are a few holes where the ice has melted.
If you step into one of those holes, you'll fall into the freezing water.
At this time, there's an international frisbee shortage, so it's absolutely imperative that
you navigate across the lake and retrieve the disc.
However, the ice is slippery, so you won't always move in the direction you intend.
The surface is described using a grid like the following
    SFFF
    FHFH
    FFFH
    HFFG
S : starting point, safe
F : frozen surface, safe
H : hole, fall to your doom
G : goal, where the frisbee is located
The episode ends when you reach the goal or fall in a hole.
You receive a reward of 1 if you reach the goal, and zero otherwise.

Action space:
LEFT = 0
DOWN = 1
RIGHT = 2
UP = 3
"""
    



In [ ]:
env = FrozenLakeEnv(map_name="1x8",is_slippery=False)
print("Environment state:")
env.render()
print("Observation space:", env.observation_space)
print("Action space:", env.action_space)
env.close()

# Scripted Interaction

In [ ]:
observation = env.reset()

env.render()
"""walk left"""
action = 0
env.step(action)
env.render()

"""walk right 3x"""
action = 2 
env.step(action)
env.render()
env.step(action)
env.render()
env.step(action)
env.render()

"""walk down  (nothing should happen, same with up)"""
env.step(1)
env.render()

"""walk right again"""
observation, reward, done, info = env.step(2)
print("Observation:", observation, "Reward:", reward, "Done:", done)
env.render()
observation, reward, done, info = env.step(2)
env.render()
print("Observation:", observation, "Reward:", reward, "Done:", done)

env.close()

# Random Interation
Run the cell below a few times and you'll notice that a random agent can take between a few steps to over 100 steps to navigate this straight line maze.

Keeping in mind this is just limiting the actions to left or right

In [ ]:
"""Run for 1 episode"""
for i_episode in range(1):
    observation = env.reset()
    for t in range(500):
        env.render()
        action = random.sample([0, 2], 1)[0] #pick 1 sample from 0 (left) or 2 (right)
        observation, reward, done, info = env.step(action)
        if done:
            print("Episode finished successfully after {} timesteps".format(t+1))
            break
env.close()

# The tools

## Epsilon ($\varepsilon$) Greedy - explore or exploit
The greek letter epsilon is used to indicate exploration rate, it's the probability that our agent will explore.

$\varepsilon$ = 1.00 $\to$ 100% explore  
$\varepsilon$ = 0.00 $\to$ 100% exploit  

Here we'll create a class that makes it easy for us to keep track of our epsilon value, with a flag (epsilon.isTraining) to indicate whether we're in training mode or in run mode.

There are many approaches to epsilon-greedy, this is just a simple way

In [4]:
class Epsilon(object):
    def __init__(self, start=1.0, end=0.01, update_increment=0.01):
        self.start = start
        self.end = end
        self.update_increment = update_increment
        self._value = self.start
        self.isTraining = True
    
    def increment(self, count=1):
        self._value = max(self.end, self._value - self.update_increment*count)
        return self
        
    def value(self):
        if not self.isTraining:
            return 0.0
        else:
            return self._value
"""
Instantiate object with epsilon starting at 1.0 (100% exploration), final value 0.01 (1% exploration), 
each time we call increment it'll go down by 0.01. 
If eps.isTraining is set to True then it'll return 0.0 (zero exploration)
"""
eps = Epsilon(start=1.0, end=0.01, update_increment=0.01)
print(eps.value())
print("Incrementing 3 times")
print(eps.increment().value())
print(eps.increment().value())
print(eps.increment().value())
print("Increment 99 times and the lowest it goes to is 0.01")
print(eps.increment(99).value())
print("Set training = False")
eps.isTraining = False
print(eps.increment().value())
print("Set training = True")
eps.isTraining = True
print(eps.increment().value())

1.0
Incrementing 3 times
0.99
0.98
0.97
Increment 99 times and the lowest it goes to is 0.01
0.01
Set training = False
0.0
Set training = True
0.01



## Implementing $Q(S_{t}, A_{t})$
This can implemented as a table where the index is the state, each record is a list of the Q values for that state which in our scenario is 2 Q values (move left and right)


| State         | Left (0)      | Right (2) |
| ------------- | -------------:| ---------:|
| state 1       |          0.45 |      0.87 |
| state 2       |          0.35 |      0.54 |
| state 3       |          0.73 |      0.34 |

The python dictionary is a good mechanism for this

In [ ]:
"""Create a dictionary as our Q table"""
Q = {}

"""Insert a single state action pair"""
print("Simple State")
s = tuple([5])
a = [90, 92]  # left and right Q values
Q[s] = a

print("s:", s)
print("Retrieving (s):", Q[s])
print("Retrieving (s, left):", Q[s][0])
print("Retrieving (s, right):", Q[s][1], "\n")

"""Insert a complex state"""
print("Complex state")
s = tuple([5, 6, 8, 9])
a = [100, 102]  # left and right Q values
Q[s] = a

print("s:", s)
print("Retrieving (s):", Q[s])
print("Retrieving (s, left):", Q[s][0])
print("Retrieving (s, right):", Q[s][1], "\n")

"""Overriding a Q value"""
print("Overriding a Q value")
print("Q before overwriting:", Q)
Q[s][0] = 101
print("Q after overwriting:", Q)
print("Retrieving (s):", Q[s])

# Exercise - Make an agent that runs the frozen lake

### Pseudocode
```
Initialize Q(s, a), for all S, A, arbitrarily  
Repeat (for each episode)  
    Initialize S  
    Repeat (for each step of episode):  
        Choose A from S using policy derived from Q (epsilon - Greedy)  
        Take action, observe R, S'
        Q(S, A) <-- Q(S,A) + alpha [R + gamma * maxQ(S', a) - Q(S,A)]
        S <-- S'
    until S is terminal
```

*S' $= S_{t+1}$*  
*maxQ(S', a) above is $\underset{a}{\operatorname{max}} Q(S_{t+1}, a)$, can't do math notation in a code block*

### Suggested progression
- Get agent training loop working with random action
- Get agent action selection to use epsilon-greedy
- Get agent to store and update Q values

In [ ]:
class Agent():
    def __init__(self):
        ## TODO
        ## Initialize Q table
        ## Initialize epsilon
        pass
    
    def getAction(self):
        action = random.sample([0, 2], 1)[0] #pick 1 sample from 0 (left) or 2 (right)
        return action
    
    def train(self):
        
        pass
    
    def run(self):
        S = env.reset()
        steps = 0
        while True:
            env.render()
            action = self.getAction()
            S_1, reward, done, info = env.step(action)
            steps += 1
            if done:
                print("Episode finished successfully after {} timesteps".format(steps+1))
                break
agent = Agent()

agent.run()

# Solution

In [23]:
class Agent():
    def __init__(self):
        self.env = FrozenLakeEnv(map_name="1x8",is_slippery=False)
        self.Q = {}
        self.epsilon = Epsilon(start=1.0, end=0.01, update_increment=0.01)
    
    def getAction(self, s):
        if np.random.rand() >= self.epsilon.value():
            action = np.argmax(self.getQ(s))
            if action == 1:
                action = 2
        else:
            action = random.sample([0, 2], 1)[0] #pick 1 sample from 0 (left) or 2 (right)
        return action
    
    def getQ(self, s, action=None):
        if not s in self.Q:
            self.Q[s] = [0, 0, 0, 0]

        if action is not None:
            return self.Q[s][action]
        else:
            return self.Q[s]
    
    def train(self, episodes=100):
        self.epsilon.isTraining = True
        # run for 100 episodes:
        for i in range(episodes):
            s = tuple([self.env.reset()])
            steps = 0
            while True:
                action = self.getAction(s)
                
                s_1, reward, done, info = self.env.step(action)
                s_1 = tuple([s_1])
                
                q = self.getQ(s, action)
                max_q_s_1 = np.max(self.getQ(s_1))
                q = q + 1.0 * (reward + 0.90 * max_q_s_1 - q)
                self.Q[s][action] = q
#                 self.env.render()
                s = s_1
                
                steps += 1
                if done:
                    print("Training episode finished after {} timesteps".format(steps))
                    break
            self.epsilon.increment(5)
                
    
    def run(self):
        print("Running agent with this Q table")
        for s in self.Q:
            print(s, self.Q[s])
        self.epsilon.isTraining = False
        s = self.env.reset()
        s = tuple([s])
        print(s)
        steps = 0
        while True:
            self.env.render()
            action = self.getAction(s)
            s_1, reward, done, info = self.env.step(action)
            s_1 = tuple([s_1])
            s = s_1
            steps += 1
            if done:
                print("Episode finished successfully after {} timesteps".format(steps))
                break


agent = Agent()
agent.train(episodes=22)
# agent.run()

Training episode finished after 6 timesteps
Training episode finished after 41 timesteps
Training episode finished after 8 timesteps
Training episode finished after 37 timesteps
Training episode finished after 22 timesteps
Training episode finished after 8 timesteps
Training episode finished after 8 timesteps
Training episode finished after 18 timesteps
Training episode finished after 12 timesteps
Training episode finished after 4 timesteps
Training episode finished after 4 timesteps
Training episode finished after 4 timesteps
Training episode finished after 10 timesteps
Training episode finished after 14 timesteps
Training episode finished after 4 timesteps
Training episode finished after 6 timesteps
Training episode finished after 4 timesteps
Training episode finished after 4 timesteps
Training episode finished after 4 timesteps
Training episode finished after 4 timesteps
Training episode finished after 4 timesteps
Training episode finished after 4 timesteps


In [24]:
agent.run()

Running agent with this Q table
(5,) [0.7290000000000001, 0, 0.9, 0]
(0,) [0.0, 0, 0.0, 0]
(6,) [0.81, 0, 1.0, 0]
(1,) [0.0, 0, 0.5904900000000002, 0]
(7,) [0, 0, 0, 0]
(2,) [0.5314410000000002, 0, 0.6561000000000001, 0]
(3,) [0.5904900000000002, 0, 0.7290000000000001, 0]
(4,) [0.6561000000000001, 0, 0.81, 0]
(3,)

FFFSFFFG
  (Right)
FFFSFFFG
  (Right)
FFFSFFFG
  (Right)
FFFSFFFG
Episode finished successfully after 4 timesteps
